# MEG/EEG Dipole Interactive Example

This notebook is an interactive example of how dipoles manifest at the scalp through electromagnetic conduction and detection via electrodes (EEG) and gradiometers and magnetometers (MEG).

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
plt.ioff()

In [3]:
import os.path as op
import numpy as np

import mne
from mne.datasets import sample

from mayavi import mlab
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

data_path = sample.data_path()
subjects_dir = subjects_dir = op.join(data_path, 'subjects')
data_dir = op.join(data_path, 'MEG', 'sample')

# setup sorce space
src = mne.setup_source_space('sample', subjects_dir=subjects_dir,
                             spacing='oct6')

# get info for forward model
fname_trans = op.join(data_dir, 'sample_audvis_raw-trans.fif')
fname_bem_eeg = op.join(subjects_dir, 'sample', 'bem',
                        'sample-5120-5120-5120-bem-sol.fif')
fname_bem_meg = op.join(subjects_dir, 'sample', 'bem',
                        'sample-5120-bem-sol.fif')

# read in evoked
fname = op.join(data_dir, 'sample_audvis-ave.fif')
evoked = mne.read_evokeds(fname, baseline=(None, 0), proj=True)[0]
info = evoked.info

fwd_eeg = mne.make_forward_solution(info, trans=fname_trans,
                                    src=src, bem=fname_bem_eeg,
                                    eeg=True, meg=False)

fwd_meg = mne.make_forward_solution(info, trans=fname_trans,
                                    src=src, bem=fname_bem_meg,
                                    meg=True, eeg=False)

# setup data
fname_stc_eeg = op.join(data_dir, 'sample_audvis-eeg-lh.stc')
stc_eeg = mne.read_source_estimate(fname_stc_eeg)
stc_eeg = stc_eeg.crop(tmax=stc.times[0])  # we just need one time point

fname_stc_meg = op.join(data_dir, 'sample_audvis-meg-lh.stc')
stc_meg = mne.read_source_estimate(fname_stc_meg)
stc_meg = stc_meg.crop(tmax=stc.times[0])  # we just need one time point

# components for GUI
strength = widgets.IntSlider(min=0, max=100, step=1, value=10)
r = widgets.FloatSlider(min=0., max=1., step=0.1, value=0.1)
phi = widgets.IntSlider(min=0, max=360, step=1)
theta = widgets.IntSlider(min=0, max=180, step=1)
x = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.)
y = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.)
z = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.)
modality = widgets.RadioButtons(
    options=['grad', 'mag', 'eeg'],
    description='Modality: ',
    disabled=False
)

plot = mlab.quiver3d(x.value, y.value, z.value,
                     *get_uvw(r.value, phi.value,
                              theta.value),
                     line_width=strength.value,
                     scale_factor=1e-2)

def get_uvw(r, theta, phi):
    u = r * np.cos(theta) * np.sin(phi)
    v = r * np.sin(theta) * np.sin(phi)
    w = r * np.cos(phi)
    return u, v, w

def set_stc_data(stc, x, y, z, u, v, w, strength):
    stc.data[:] = 0
    
    return stc

ui = widgets.HBox([strength, r, phi, theta, x, y, z, modality])
def update_dipole(strength, r, phi, theta, x, y, z, modality):
    u, v, w = get_uvw(r, phi, theta)
    # TO DO: get source space locations, create vector,
    # assign source space activity based on vector,
    # plot source space
    if modality == 'eeg':
        stc_eeg = set_stc_data(stc_eeg, x, y, z, u, v, w, strength)
        evoked = mne.forward.apply_forward(fwd_eeg, stc_eeg,
                                           info, on_missing='ignore')
    else:
        stc_meg = set_stc_data(stc_meg, x, y, z, u, v, w, strength)
        evoked = mne.forward.apply_forward(fwd_meg, stc_meg,
                                           info, on_missing='ignore')
    maps = mne.make_field_map(evoked, trans=fname_trans, subject='sample',
                              subjects_dir=subjects_dir, n_jobs=1)
    field_map = evoked.plot_field(maps)
    plot.mlab_source.trait_set(x=x, y=y, z=z, u=u, v=v, w=w,
                               line_width=strenght)
    

out = widgets.interactive_output(update_dipole,
                                 {'strength': strength,
                                  'r': r, 'phi': phi, 'theta': theta,
                                  'x': x, 'y': y, 'z':, z,
                                  'modality': modality})

display(ui, out)

Reading /tmp/temp-ave.fif ...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -199.80 ...     499.49 ms (Left Auditory)
        0 CTF compensation matrices available
        nave = 55 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Applying baseline correction (mode: mean)
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -199.80 ...     499.49 ms (Right Auditory)
        0 CTF compensation matrices available
        nave = 61 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Applying baseline correction (mode: mean)
    Read a total of 4 projection 